In [3]:
from sklearn.model_selection import train_test_split
import pandas as pd

df = pd.read_csv("500 เมนู label แล้ว - Sheet1.csv")

train_val_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

train_df, val_df = train_test_split(train_val_df, test_size=0.25, random_state=42)


train_df.to_csv("train_set.csv", index=False, encoding='utf-8-sig')
val_df.to_csv("val_set.csv", index=False, encoding='utf-8-sig')
test_df.to_csv("test_set.csv", index=False, encoding='utf-8-sig')

print("Data has been split and saved to train_set.csv, val_set.csv, and test_set.csv.")


Data has been split and saved to train_set.csv, val_set.csv, and test_set.csv.


In [4]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 13.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == 

ตอนแรกอันนี้รันได้ แต่พอเปิดมาอีกรอบรันไม่ได้แล้ว จำได้ว่าตอนนั้นเทรนไปได้ acurracy ประมาณ 0.6

In [3]:
#ori dont delete why is not workinggg
from datasets import Dataset, DatasetDict
import pandas as pd
from sentence_transformers import SentenceTransformer, losses, InputExample, models, SentencesDataset
from sentence_transformers.evaluation import LabelAccuracyEvaluator
from torch.utils.data import DataLoader
import torch
from sklearn.preprocessing import LabelEncoder

train_df = pd.read_csv("train_set.csv")
val_df = pd.read_csv("val_set.csv")
test_df = pd.read_csv("test_set.csv")

#Labelอาหาร
all_data = pd.concat([train_df['ชื่ออาหาร']])
label_encoder = LabelEncoder()
label_encoder.fit(all_data)

train_df['label'] = label_encoder.transform(train_df['ชื่ออาหาร'])

train_examples = [
    InputExample(texts=[row['วัตถุดิบ']], label=int(row['label']))
    for _, row in train_df.iterrows()
]

val_examples = [
    InputExample(texts=[row['วัตถุดิบ']], label=int(row['label']))
    for _, row in train_df.iterrows()
]

model= SentenceTransformer('paraphrase-multilingual-mpnet-base-v2')
# Define sentences before using them
sentences = train_df['วัตถุดิบ'].tolist()
pool = model.start_multi_process_pool()
embeddings = model.encode_multi_process(sentences, pool)
model.stop_multi_process_pool(pool)

word_embedding_model = model._first_module()
pooling_model = model._last_module()
dense_model = models.Dense(in_features=pooling_model.get_sentence_embedding_dimension(), out_features=len(label_encoder.classes_), activation_function=torch.nn.Tanh())


model = SentenceTransformer(modules=[word_embedding_model, pooling_model, dense_model])

train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)
train_loss = losses.SoftmaxLoss(
    model=model,
    sentence_embedding_dimension=pooling_model.get_sentence_embedding_dimension(),
    num_labels=len(label_encoder.classes_)
)


val_dataloader = DataLoader(val_examples, batch_size=16)
evaluator = LabelAccuracyEvaluator(
    dataloader=val_dataloader,
    softmax_model=train_loss
)


model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    evaluator=evaluator,
    epochs=3,
    evaluation_steps=100,
    warmup_steps=10,
    output_path='output/sbert-food-classifier'
)

# Save label mapping
import pickle
with open('output/label_encoder.pkl', 'wb') as f:
    pickle.dump(label_encoder, f)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.90k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

ArrowTypeError: Expected bytes, got a 'float' object

อันนี้คือแก้จากอันด้านบน softmaxloss


In [4]:
from datasets import Dataset, DatasetDict
import pandas as pd
from sentence_transformers import SentenceTransformer, losses, InputExample, models, SentencesDataset
from sentence_transformers.evaluation import LabelAccuracyEvaluator
from torch.utils.data import DataLoader
import torch
from sklearn.preprocessing import LabelEncoder
import pickle

# Load datasets
train_df = pd.read_csv("train_set.csv")
val_df = pd.read_csv("val_set.csv")
test_df = pd.read_csv("test_set.csv")

# Drop rows with missing or non-string ingredients
train_df = train_df.dropna(subset=['วัตถุดิบ', 'ชื่ออาหาร'])
val_df = val_df.dropna(subset=['วัตถุดิบ', 'ชื่ออาหาร'])

train_df['วัตถุดิบ'] = train_df['วัตถุดิบ'].astype(str)
val_df['วัตถุดิบ'] = val_df['วัตถุดิบ'].astype(str)

# Label encoding
all_data = pd.concat([train_df['ชื่ออาหาร'], val_df['ชื่ออาหาร']])
label_encoder = LabelEncoder()
label_encoder.fit(all_data)

train_df['label'] = label_encoder.transform(train_df['ชื่ออาหาร'])
val_df['label'] = label_encoder.transform(val_df['ชื่ออาหาร'])

# Create InputExamples
train_examples = [
    InputExample(texts=[row['วัตถุดิบ'], row['ชื่ออาหาร']], label=int(row['label'])) # Include both 'วัตถุดิบ' and 'ชื่ออาหาร' as input texts
    for _, row in train_df.iterrows()
]

val_examples = [
    InputExample(texts=[row['วัตถุดิบ'], row['ชื่ออาหาร']], label=int(row['label']))  # Include both 'วัตถุดิบ' and 'ชื่ออาหาร' as input texts
    for _, row in val_df.iterrows()
]

# Load base model
model = SentenceTransformer('paraphrase-multilingual-mpnet-base-v2')

# Add custom dense layer
word_embedding_model = model._first_module()
pooling_model = model._last_module()
dense_model = models.Dense(
    in_features=pooling_model.get_sentence_embedding_dimension(),
    out_features=len(label_encoder.classes_),
    activation_function=torch.nn.Tanh()
)
model = SentenceTransformer(modules=[word_embedding_model, pooling_model, dense_model])

# Prepare data loaders
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)
val_dataloader = DataLoader(val_examples, shuffle=False, batch_size=16)

# Define loss
train_loss = losses.SoftmaxLoss(
    model=model,
    sentence_embedding_dimension=model.get_sentence_embedding_dimension(),
    num_labels=len(label_encoder.classes_)
)

# Define evaluator
evaluator = LabelAccuracyEvaluator(
    dataloader=val_dataloader,
    softmax_model=train_loss
)

# Train model
model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    evaluator=evaluator,
    epochs=3,
    evaluation_steps=100,
    warmup_steps=10,
    output_path='output/sbert-food-classifier',
    show_progress_bar=True
)

# Save label encoder
with open('output/label_encoder.pkl', 'wb') as f:
    pickle.dump(label_encoder, f)


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: chanisorn-siri (chanisorn-siri-bodindecha) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss,Validation Loss,Accuracy
19,No log,No log,0.000000
38,No log,No log,0.000000
57,No log,No log,0.000000


อันนี้ใช้ MultipleNegativesRankingLoss


In [6]:
from datasets import Dataset, DatasetDict
import pandas as pd
from sentence_transformers import (
    SentenceTransformer,
    SentenceTransformerTrainer,
    SentenceTransformerTrainingArguments,
    SentenceTransformerModelCardData,
)
from sentence_transformers.losses import MultipleNegativesRankingLoss
from sentence_transformers.training_args import BatchSamplers
from sentence_transformers.evaluation import SentenceEvaluator
from sklearn.model_selection import train_test_split

# Load CSV
df = pd.read_csv("500 เมนู label แล้ว - Sheet1.csv")

# Rename columns for simplicity
df = df.rename(columns={"วัตถุดิบ": "ingredients", "ชื่ออาหาร": "dish"})

# Drop missing values in important columns
df = df.dropna(subset=["ingredients", "dish"])

# Split into train/val/test
train_df, temp_df = train_test_split(df, test_size=0.2, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

# Convert to Hugging Face Dataset
train_dataset = Dataset.from_pandas(train_df[["ingredients", "dish"]])
val_dataset = Dataset.from_pandas(val_df[["ingredients", "dish"]])
test_dataset = Dataset.from_pandas(test_df[["ingredients", "dish"]])

# Model loading
model = SentenceTransformer(
    'paraphrase-multilingual-mpnet-base-v2',
    model_card_data=SentenceTransformerModelCardData(
        language="th",
        license="apache-2.0",
        model_name="Thai Food Ingredients → Dish Prediction",
    )
)

# MultipleNegativesRankingLoss requires only anchor-positive pairs
train_samples = list(zip(train_dataset["ingredients"], train_dataset["dish"]))
val_samples = list(zip(val_dataset["ingredients"], val_dataset["dish"]))
test_samples = list(zip(test_dataset["ingredients"], test_dataset["dish"]))

# Convert samples into DatasetDict with correct format
def convert_to_dict(samples):
    return Dataset.from_dict({
        "anchor": [a for a, b in samples],
        "positive": [b for a, b in samples],
    })

train_dataset = convert_to_dict(train_samples)
val_dataset = convert_to_dict(val_samples)
test_dataset = convert_to_dict(test_samples)

# loss
loss = MultipleNegativesRankingLoss(model)

# training arguments
args = SentenceTransformerTrainingArguments(
    output_dir="models/thai-food-mpnet",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    learning_rate=2e-5,
    warmup_ratio=0.1,
    fp16=True,
    bf16=False,
    batch_sampler=BatchSamplers.NO_DUPLICATES,
    eval_strategy="steps",
    eval_steps=50,
    save_strategy="steps",
    save_steps=100,
    save_total_limit=2,
    logging_steps=50,
    run_name="thai-food-retriever",
)

# Define dummy evaluator (optional)
from sentence_transformers.evaluation import InformationRetrievalEvaluator
evaluator = None

# trainer
trainer = SentenceTransformerTrainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    loss=loss,
    evaluator=evaluator
)

# Train
trainer.train()

# Save the final model
model.save_pretrained("models/thai-food-mpnet2")


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

Step,Training Loss,Validation Loss


Step,Training Loss,Validation Loss
50,1.257800,0.767191


In [12]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import numpy as np

model = SentenceTransformer("models/thai-food-mpnet2")

df = pd.read_csv("500 เมนู label แล้ว - Sheet1.csv")
df = df.rename(columns={"วัตถุดิบ": "ingredients", "ชื่ออาหาร": "dish"})
df = df.dropna(subset=["ingredients", "dish"])

ingredient_embeddings = model.encode(val_dataset["anchor"], convert_to_tensor=True)
dish_embeddings = model.encode(val_dataset["positive"], convert_to_tensor=True)

cos_sim = cosine_similarity(ingredient_embeddings.cpu(), dish_embeddings.cpu())

top1_correct = 0
for i in range(len(cos_sim)):
    if np.argmax(cos_sim[i]) == i:
        top1_correct += 1

accuracy = top1_correct / len(cos_sim)
print(f"Top-1 Matching Accuracy on Validation Set: {accuracy:.4f}")

dish_texts = df["dish"].tolist()
dish_embeddings = model.encode(dish_texts, convert_to_tensor=True)

input_sets = [
    "ไข่ หมู ไก่",
    "น้ำมัน แป้ง",
    "กล้วย น้ำตาล"
]

for user_input in input_sets:
    input_embedding = model.encode([user_input], convert_to_tensor=True)
    cos_sim = cosine_similarity(input_embedding.cpu(), dish_embeddings.cpu())[0]

    top_index = np.argmax(cos_sim)
    matched_dish = df.iloc[top_index]["dish"]
    matched_ingredients = df.iloc[top_index]["ingredients"]
    similarity_score = cos_sim[top_index]

    print(f"\n🔍 Input Ingredients: {user_input}")
    print(f"✅ Matched Dish: {matched_dish}")
    print(f"🧂 Ingredients: {matched_ingredients}")
    print(f"📏 Similarity Score: {similarity_score:.4f}")


Top-1 Matching Accuracy on Validation Set: 0.5918

🔍 Input Ingredients: ไข่ หมู ไก่
✅ Matched Dish: ไข่เจียวกากหมู
🧂 Ingredients: ไข่เป็ด:, 2 ฟอง
กากหมูทอด: ตามชอบ
น้ำปลา:, 1 ช้อนโต๊ะ
พริกไทย: ตามชอบ
📏 Similarity Score: 0.8683

🔍 Input Ingredients: น้ำมัน แป้ง
✅ Matched Dish: ขนมอินทนิล
🧂 Ingredients: แป้งมันสำปะหลัง, 100 กรัม
น้ำใบเตยเข้มข้น, 500 มล.
กะทิเรียลไทย, 250 มล.
น้ำลอยดอกมะลิ หรือหางกะทิ, 100 มล.
น้ำตาลทราย, 90 กรัม
เกลือ, 1/2 ช้อนชา
ใบเตย
เทียนอบขนม
📏 Similarity Score: 0.6310

🔍 Input Ingredients: กล้วย น้ำตาล
✅ Matched Dish: กล้วยน้ำว้าอบ หวานจากธรรมชาติ
🧂 Ingredients: กล้วย, 1 หวี
📏 Similarity Score: 0.8148


In [5]:
from datasets import Dataset, DatasetDict
import pandas as pd
from sentence_transformers import (
    SentenceTransformer,
    SentenceTransformerTrainer,
    SentenceTransformerTrainingArguments,
    SentenceTransformerModelCardData,
)
from sentence_transformers.losses import MultipleNegativesRankingLoss
from sentence_transformers.training_args import BatchSamplers
from sentence_transformers.evaluation import SentenceEvaluator
from sklearn.model_selection import train_test_split

# Load CSV
df = pd.read_csv("500 เมนู label แล้ว - Sheet1.csv")

# Rename columns for simplicity
df = df.rename(columns={"วัตถุดิบ": "ingredients", "ชื่ออาหาร": "dish"})

# Drop missing values in important columns
df = df.dropna(subset=["ingredients", "dish"])

# Split into train/val/test
train_df, temp_df = train_test_split(df, test_size=0.2, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

# Convert to Hugging Face Dataset
train_dataset = Dataset.from_pandas(train_df[["ingredients", "dish"]])
val_dataset = Dataset.from_pandas(val_df[["ingredients", "dish"]])
test_dataset = Dataset.from_pandas(test_df[["ingredients", "dish"]])

# Model loading
model = SentenceTransformer(
    'paraphrase-multilingual-mpnet-base-v2',
    model_card_data=SentenceTransformerModelCardData(
        language="th",
        license="apache-2.0",
        model_name="Thai Food Ingredients → Dish Prediction",
    )
)

# MultipleNegativesRankingLoss requires only anchor-positive pairs
train_samples = list(zip(train_dataset["ingredients"], train_dataset["dish"]))
val_samples = list(zip(val_dataset["ingredients"], val_dataset["dish"]))
test_samples = list(zip(test_dataset["ingredients"], test_dataset["dish"]))

# Convert samples into DatasetDict with correct format
def convert_to_dict(samples):
    return Dataset.from_dict({
        "anchor": [a for a, b in samples],
        "positive": [b for a, b in samples],
    })

train_dataset = convert_to_dict(train_samples)
val_dataset = convert_to_dict(val_samples)
test_dataset = convert_to_dict(test_samples)

# loss
loss = MultipleNegativesRankingLoss(model)

# training arguments
args = SentenceTransformerTrainingArguments(
    output_dir="models/thai-food-mpnet-fast",
    num_train_epochs=2,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    learning_rate=2e-5,
    warmup_ratio=0.1,
    fp16=False,
    bf16=False,
    batch_sampler=BatchSamplers.NO_DUPLICATES,
    eval_strategy="steps",
    eval_steps=100,
    save_strategy="steps",
    save_steps=200,
    save_total_limit=1,
    logging_steps=100,
    run_name="thai-food-retriever-fast",
)

# Define dummy evaluator (optional)
from sentence_transformers.evaluation import InformationRetrievalEvaluator
evaluator = None

# trainer
trainer = SentenceTransformerTrainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    loss=loss,
    evaluator=evaluator
)

# Train
trainer.train()

# Save the final model
model.save_pretrained("models/thai-food-mpnet/final")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.90k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: chanisorn-siri (chanisorn-siri-bodindecha) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss,Validation Loss


In [14]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import numpy as np

model = SentenceTransformer("models/thai-food-mpnet/final")

df = pd.read_csv("500 เมนู label แล้ว - Sheet1.csv")
df = df.rename(columns={"วัตถุดิบ": "ingredients", "ชื่ออาหาร": "dish"})
df = df.dropna(subset=["ingredients", "dish"])

ingredient_embeddings = model.encode(val_dataset["anchor"], convert_to_tensor=True)
dish_embeddings = model.encode(val_dataset["positive"], convert_to_tensor=True)

cos_sim = cosine_similarity(ingredient_embeddings.cpu(), dish_embeddings.cpu())

top1_correct = 0
for i in range(len(cos_sim)):
    if np.argmax(cos_sim[i]) == i:
        top1_correct += 1

accuracy = top1_correct / len(cos_sim)
print(f"Top-1 Matching Accuracy on Validation Set: {accuracy:.4f}")

dish_texts = df["dish"].tolist()
dish_embeddings = model.encode(dish_texts, convert_to_tensor=True)

input_sets = [
    "ไข่ หมู ไก่",
    "น้ำมัน แป้ง",
    "กล้วย น้ำตาล"
]

for user_input in input_sets:
    input_embedding = model.encode([user_input], convert_to_tensor=True)
    cos_sim = cosine_similarity(input_embedding.cpu(), dish_embeddings.cpu())[0]

    top_index = np.argmax(cos_sim)
    matched_dish = df.iloc[top_index]["dish"]
    matched_ingredients = df.iloc[top_index]["ingredients"]
    similarity_score = cos_sim[top_index]

    print(f"\n🔍 Input Ingredients: {user_input}")
    print(f"✅ Matched Dish: {matched_dish}")
    print(f"🧂 Ingredients: {matched_ingredients}")
    print(f"📏 Similarity Score: {similarity_score:.4f}")


Top-1 Matching Accuracy on Validation Set: 0.5714

🔍 Input Ingredients: ไข่ หมู ไก่
✅ Matched Dish: ไข่เจียวกากหมู
🧂 Ingredients: ไข่เป็ด:, 2 ฟอง
กากหมูทอด: ตามชอบ
น้ำปลา:, 1 ช้อนโต๊ะ
พริกไทย: ตามชอบ
📏 Similarity Score: 0.8988

🔍 Input Ingredients: น้ำมัน แป้ง
✅ Matched Dish: ปังกรอบเนยน้ำตาล
🧂 Ingredients: ขนมปังแผ่น, 1 แถว
เนยสด, 1ก้อน หรือประมาณ 227 กรัม
น้ำตาลทรายโรยหน้าตามความชอบ
📏 Similarity Score: 0.7041

🔍 Input Ingredients: กล้วย น้ำตาล
✅ Matched Dish: กล้วยน้ำว้าอบ หวานจากธรรมชาติ
🧂 Ingredients: กล้วย, 1 หวี
📏 Similarity Score: 0.8071


In [16]:
import shutil
from google.colab import files

# Zip the model folder
shutil.make_archive('thai-food-mpnet2', 'zip', 'models/thai-food-mpnet2')

# Download the zip file to your local machine
files.download('thai-food-mpnet2.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [18]:
import shutil
from google.colab import files

shutil.make_archive('thai-food-mpnet/final', 'zip', 'models/thai-food-mpnet/final')

files.download('thai-food-mpnet/final.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>